# Neural Network Model for MNIST Dataset

### 패키지

In [1]:
import numpy as np

import tensorflow as tf

import keras
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

from sklearn.manifold import TSNE
from skimage import io

import theano

import json

Using TensorFlow backend.


In [2]:
# Functions 
def get_hot_idx(arr):
    return arr.index(max(arr))

def save_to_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    print(filename + ' 저장완료')
    
def get_round_array(array, decimal):
    return [round(e, decimal) for e in array]

def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    activations = get_activations([X_batch,0])
    return activations

def get_arr_from_json_file(filename):
    input_file = open (filename)
    return json.load(input_file)
    
current_milli_time = lambda: int(round(time.time() * 1000))

### 데이터셋 로드

In [3]:
image_width = 28
image_height = 28
num_of_features = image_width * image_height

num_of_class = 10
num_of_trainset = 60000
num_of_testset = 10000

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(num_of_trainset, num_of_features).astype('float32') / 255.0
X_test = X_test.reshape(num_of_testset, num_of_features).astype('float32') / 255.0
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [33]:
# 로컬 테스트 데이터 로드
images = np.zeros((10000, 784))
for real in range(10):
    for idx in range(1, 1001):
        file = '../../data/mnist/images/'+ str(real) + '/' + str(real) + '_' + str(idx) +'.png'
        image = np.ndarray.flatten(io.imread(file)) / 255.0
        image = np.array([1 - pixel for pixel in image])
        images[real * 1000 + idx - 1] = image

### 모델 구성

In [37]:
# 모델 구축 
model = Sequential()
model.add(Dense(units=128, input_dim=num_of_features, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=num_of_class, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### 모델 학습

In [38]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
 1472/60000 [..............................] - ETA: 1:12 - loss: 2.2968 - acc: 0.1447  ETA: 23:20 - loss: 2.3181 - acc: 0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.137419). Check your callbacks.
  % delta_t_median)


60000/60000 [==============================] - 10s 166us/step - loss: 1.2879 - acc: 0.5706
Epoch 2/10
60000/60000 [==============================] - 9s 142us/step - loss: 0.4151 - acc: 0.8832
Epoch 3/10
60000/60000 [==============================] - 10s 159us/step - loss: 0.2645 - acc: 0.92530s - loss: 0.2679 - acc: 0.924 - ETA: 0s - loss: 0.2677 -
Epoch 4/10
60000/60000 [==============================] - ETA: 0s - loss: 0.2058 - acc: 0.9414- ETA: 5s - loss: 0.21 - E - ETA: 2s - loss: - 10s 173us/step - loss: 0.2054 - acc: 0.9416
Epoch 5/10
60000/60000 [==============================] - 9s 147us/step - loss: 0.1682 - acc: 0.9523
Epoch 6/10
60000/60000 [==============================] - 11s 180us/step - loss: 0.1446 - acc: 0.9580
Epoch 7/10
60000/60000 [==============================] - 11s 177us/step - loss: 0.1273 - acc: 0.96378s -  - ETA: 0s - loss: 0.1278 - a
Epoch 8/10
60000/60000 [==============================] - 10s 167us/step - loss: 0.1108 - acc: 0.9676
Epoch 9/10
60000/60000 

### 테스트 셋 결과 확인

In [22]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print('loss_and_metrics : ' + str(loss_and_metrics))

10000/10000 [==============================] - 0s 48us/step
loss_and_metrics : [0.14955389682762324, 0.9558]


## 내 데이터로 테스트

In [35]:
# 예측
pred_proba =  model.predict_proba(images).tolist()

In [36]:
# 성능확인 
correct = 0
res = []
for i in range(len(pred_proba)):
    prob = pred_proba[i]
    pred = get_hot_idx(prob)
    real = i // 1000
    res.append([real, pred, real is pred, i])
    if pred is real:
        correct = correct + 1
print(correct)

9513


### 차원축소

1) 마지막 직전의 레이어에서 activation values를 뽑는다.

2) t-SNE로 차원을 축소한다.

In [199]:
samples = []
idxs = get_arr_from_json_file('./sample_image_idxs.json')
for idx in idxs:
    samples.append(images[idx])
print(len(samples))

500


In [200]:
extracted_features = get_activations(model, -2, samples)[0]
y = TSNE(n_components=2).fit_transform(extracted_features)